<a href="https://colab.research.google.com/github/agu3/ds4bmeTest/blob/master/a8_q2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading and preprocessing data

In [0]:
!pip install -U tensorflow_datasets

Requirement already up-to-date: tensorflow_datasets in /usr/local/lib/python3.6/dist-packages (3.0.0)


In [0]:
import tensorflow as tf
import tensorflow_datasets as tdfs
import math
import numpy as np 
import matplotlib.pyplot as plt

import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

In [0]:
fashmnist, metadata = tdfs.load('fashion_mnist', as_supervised = True, with_info = True)
train_dataset, test_dataset = fashmnist['train'], fashmnist['test']

class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal',      'Shirt',   'Sneaker',  'Bag',   'Ankle boot']

In [0]:
num_train_examples = metadata.splits['train'].num_examples 
num_test_examples = metadata.splits['test'].num_examples

print(num_test_examples)
print(num_train_examples)

10000
60000


In [0]:
def normalize (images, labels):
  images = tf.cast(images, tf.float32)
  images /= 255
  return images, labels

train_dataset = train_dataset.map(normalize)
test_datatset = test_dataset.map(normalize)

train_dataset = train_dataset.cache()
test_dataset = test_dataset.cache()

# Model 1: Baseline model with 1 hidden layer with 128 nodes and 20% dropout

In [0]:
model1 = tf.keras.Sequential([
  tf.keras.layers.Flatten(input_shape = (28, 28, 1)),
  tf.keras.layers.Dense(128, activation = tf.nn.relu),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

In [0]:
model1.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 128)               100480    
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 10)                1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


In [0]:
model1.compile(
    optimizer = 'adam',
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True),
    metrics = ['accuracy']
)

In [0]:
batch_size = 32 
train_dataset = train_dataset.cache().repeat().shuffle(num_train_examples).batch(batch_size)
test_dataset = test_dataset.cache().batch(batch_size)

In [0]:
model1.fit(train_dataset, epochs = 5, steps_per_epoch = math.ceil(num_train_examples/batch_size))

Epoch 1/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.5315 - accuracy: 0.8095
Epoch 2/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.3983 - accuracy: 0.8551
Epoch 3/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.3656 - accuracy: 0.8663
Epoch 4/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.3445 - accuracy: 0.8734
Epoch 5/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.3293 - accuracy: 0.8783


In [0]:
model1.evaluate(test_dataset, steps = math.ceil(num_test_examples/batch_size))

313/313 [==============================] - 1s 3ms/step - loss: 52.5952 - accuracy: 0.8519


[52.59524154663086, 0.8518999814987183]

Accuracy with model 1 on the test dataset is 0.8518999814987183

# Model 2 with 2 hidden layers and 20% dropout

In [0]:
model2 = tf.keras.Sequential([
  tf.keras.layers.Flatten(input_shape = (28, 28, 1)),
  tf.keras.layers.Dense(128, activation = tf.nn.relu),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(64, activation = tf.nn.relu),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

In [0]:
model2.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               100480    
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                650       
Total params: 109,386
Trainable params: 109,386
Non-trainable params: 0
__________________________________________________

In [0]:
model2.compile(
    optimizer = 'adam',
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True),
    metrics = ['accuracy']
)

In [0]:
batch_size = 32 
train_dataset = train_dataset.cache().repeat().shuffle(num_train_examples).batch(batch_size)
test_dataset = test_dataset.cache().batch(batch_size)

In [0]:
model2.fit(train_dataset, epochs = 5, steps_per_epoch = math.ceil(num_train_examples/batch_size))

Epoch 1/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.5807 - accuracy: 0.7914
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.4224 - accuracy: 0.8476
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3908 - accuracy: 0.8594
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3692 - accuracy: 0.8658
Epoch 5/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3554 - accuracy: 0.8710


In [0]:
model2.evaluate(test_dataset, steps = math.ceil(num_test_examples/batch_size))

313/313 [==============================] - 1s 3ms/step - loss: 52.9044 - accuracy: 0.8415


[52.90438461303711, 0.8414999842643738]

Model 2 accuracy on the test dataset is 0.8414999842643738



# Model 3 with 3 hidden layers and 20% dropout 

In [0]:
model3 = tf.keras.Sequential([
  tf.keras.layers.Flatten(input_shape = (28, 28, 1)),
  tf.keras.layers.Dense(128, activation = tf.nn.relu),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(64, activation = tf.nn.relu),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(32, activation = tf.nn.relu),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

In [0]:
model3.summary()

In [0]:
model3.compile(
    optimizer = 'adam',
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True),
    metrics = ['accuracy']
)

In [0]:
batch_size = 32 
train_dataset = train_dataset.cache().repeat().shuffle(num_train_examples).batch(batch_size)
test_dataset = test_dataset.cache().batch(batch_size)

In [0]:
model3.fit(train_dataset, epochs = 5, steps_per_epoch = math.ceil(num_train_examples/batch_size))

In [0]:
model3.evaluate(test_dataset, steps = math.ceil(num_test_examples/batch_size))

Accuracy on the test dataset using model 3 is 0.8199999928474426

# Model 4: 2 hidden layers with 15% dropout 

In [0]:
model4 = tf.keras.Sequential([
  tf.keras.layers.Flatten(input_shape = (28, 28, 1)),
  tf.keras.layers.Dense(128, activation = tf.nn.relu),
  tf.keras.layers.Dropout(0.15),
  tf.keras.layers.Dense(64, activation = tf.nn.relu),
  tf.keras.layers.Dropout(0.15),
  tf.keras.layers.Dense(10)
])

In [0]:
model4.summary()

In [0]:
model4.compile(
    optimizer = 'adam',
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True),
    metrics = ['accuracy'] 
)

In [0]:
batch_size = 32 
train_dataset = train_dataset.cache().repeat().shuffle(num_train_examples).batch(batch_size)
test_dataset = test_dataset.cache().batch(batch_size)

In [0]:
model4.fit(train_dataset, epochs = 5, steps_per_epoch = math.ceil(num_train_examples/batch_size))

Epoch 1/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.5466 - accuracy: 0.8026
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.4081 - accuracy: 0.8523
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3777 - accuracy: 0.8603
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3544 - accuracy: 0.8713
Epoch 5/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3333 - accuracy: 0.8783


In [0]:
model4.evaluate(test_dataset, steps = math.ceil(num_test_examples/batch_size))

313/313 [==============================] - 1s 3ms/step - loss: 48.8365 - accuracy: 0.8553


[48.83653259277344, 0.8553000092506409]

Accuracy on the test dataset using model 4 is 0.8553000092506409

# Model 5: 2 hidden layers with 25% dropout

In [0]:
model5 = tf.keras.Sequential([
  tf.keras.layers.Flatten(input_shape = (28, 28, 1)),
  tf.keras.layers.Dense(128, activation = tf.nn.relu),
  tf.keras.layers.Dropout(0.25),
  tf.keras.layers.Dense(64, activation = tf.nn.relu),
  tf.keras.layers.Dropout(0.25),
  tf.keras.layers.Dense(10)
])

In [0]:
model5.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_6 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_16 (Dense)             (None, 128)               100480    
_________________________________________________________________
dropout_10 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_17 (Dense)             (None, 64)                8256      
_________________________________________________________________
dropout_11 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_18 (Dense)             (None, 10)                650       
Total params: 109,386
Trainable params: 109,386
Non-trainable params: 0
________________________________________________

In [0]:
model5.compile(
    optimizer = 'adam',
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True),
    metrics = ['accuracy'] 
)

In [0]:
batch_size = 32 
train_dataset = train_dataset.cache().repeat().shuffle(num_train_examples).batch(batch_size)
test_dataset = test_dataset.cache().batch(batch_size)

In [0]:
model5.fit(train_dataset, epochs = 5, steps_per_epoch = math.ceil(num_train_examples/batch_size))

Epoch 1/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.6152 - accuracy: 0.7836
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.4428 - accuracy: 0.8401
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.4072 - accuracy: 0.8532
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3802 - accuracy: 0.8624
Epoch 5/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3692 - accuracy: 0.8662


In [0]:
model5.evaluate(test_dataset, steps = math.ceil(num_test_examples/batch_size))

313/313 [==============================] - 1s 3ms/step - loss: 53.2220 - accuracy: 0.8497


[53.22195816040039, 0.8496999740600586]

Accuracy on the test dataset using model 5 is 0.8496999740600586